In [ ]:
import os
import pandas as pd

from PIL import Image

import plotly.express as px

In [ ]:
# This will convert MySQL database datas into CSV's and stores it

!python DB_to_CSV.py

In [ ]:
if not os.path.isfile('atliq_hardware_db.csv'):
    
    print("Failed to find 'atliq_hardware_db.csv', Reading all databases")
    fact_manufacturing_cost = pd.read_csv('DB_CSVs/fact_manufacturing_cost.csv')
    dim_product = pd.read_csv('DB_CSVs/dim_product.csv')
    fact_pre_invoice_deductions = pd.read_csv('DB_CSVs/fact_pre_invoice_deductions.csv')
    fact_sales_monthly = pd.read_csv('DB_CSVs/fact_sales_monthly.csv')
    dim_customer = pd.read_csv('DB_CSVs/dim_customer.csv')
    fact_gross_price = pd.read_csv('DB_CSVs/fact_gross_price.csv')
    
    sales_cus = pd.merge(fact_sales_monthly, dim_customer,on = 'customer_code')
    sale_cus_pro = pd.merge(sales_cus, dim_product, on = 'product_code')
    sale_cus_pro_manCost = pd.merge(sale_cus_pro, fact_manufacturing_cost.rename(columns = {'cost_year' : 'fiscal_year'}), on = ['product_code', 'fiscal_year'])
    sale_cus_pro_manCost_grossP = pd.merge(sale_cus_pro_manCost, fact_gross_price, on = ['product_code', 'fiscal_year'])
    merged_db = pd.merge(sale_cus_pro_manCost_grossP, fact_pre_invoice_deductions, on = ['customer_code', 'fiscal_year'])

    columns_order = ['date', 'customer_code', 'customer', 'product_code', 'product', 'division', 'segment', 'category', 'variant',
    'platform', 'channel', 'market', 'sub_zone', 'region', 'sold_quantity', 'manufacturing_cost',
    'gross_price', 'pre_invoice_discount_pct', 'fiscal_year']

    atliq_hardware_db = merged_db[columns_order]
    atliq_hardware_db.to_csv('atliq_hardware_db.csv', index = False)    

else:
    print("Reading from 'atliq_hardware_db.csv'")
    atliq_hardware_db = pd.read_csv('atliq_hardware_db.csv')
    
def regain_ch(row):
    if row == 'APAC': return 'Asia Pacific'
    elif row == 'EU': return 'European Union'
    elif row == 'LATAM': return 'Latin America'
    else: return 'Northern America'
    
atliq_hardware_db['region'] = atliq_hardware_db.region.apply(regain_ch)

print(atliq_hardware_db.shape)
atliq_hardware_db.head(2)


In [ ]:
atliq_hardware_db['gross_manufacturing_cost'] = atliq_hardware_db['manufacturing_cost'] * atliq_hardware_db['sold_quantity']
atliq_hardware_db['gross_sales'] = atliq_hardware_db['gross_price'] * atliq_hardware_db['sold_quantity']
atliq_hardware_db['gross_profit'] = atliq_hardware_db['gross_sales'] - atliq_hardware_db['gross_manufacturing_cost']

In [ ]:
products_2020 = atliq_hardware_db[atliq_hardware_db.fiscal_year == 2020].product_code.unique()

atliq_hardware_db['existing_products'] = atliq_hardware_db.product_code.apply(lambda x : 'no' if x in products_2020 else 'yes')

atliq_hardware_db.head(2)


In [ ]:
# https://community.plotly.com/t/plotly-express-with-xaxis-having-integers-strings/34777

img = Image.open('images/curly-braces-l.png')

fig = px.histogram(data_frame = atliq_hardware_db,
                   x = 'fiscal_year' , y = 'gross_profit',
                   color  = 'existing_products', text_auto = '.5s',
                   width = 950, height = 650,
                   title = '<b>Profit generation growth</b><br><sub>Introduction of new products in <b>2021</b> helped to gain additional <b>120.86%</b> growth</sub>')

fig.update_layout(xaxis_title = 'Financial Year',
                  yaxis_title = 'Generated Profit',
                  legend_title = 'Products',
                  xaxis_type = 'category',
                  template = 'plotly_white')

# https://stackoverflow.com/a/64378982
newnames = {'no' : 'Exisiting Productes', 'yes' : 'New products'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

fig.add_layout_image(dict(source = img, opacity = 0.3,
                          xref = 'paper', yref = 'paper',
                          sizing = 'stretch', layer = 'below'))

fig.update_layout_images(dict(xref = 'paper', yref = 'paper',
                              sizex = 0.04, sizey = 0.35,
                              xanchor ='center', yanchor ='middle',
                              x = 0.5, y = 0.76))

fig.add_annotation(text = 'Profit generated by the<br>new products alone',
                   align = 'center', xref = 'paper', yref = 'paper',
                   x = 0.25, y = 0.82, showarrow = False, 
                   font = dict(size = 13, color = 'black'))

fig.write_image('images/plots/profit_generated.png', scale = 3)

fig.show()

![](images/plots/profit_generated.png)

In [ ]:
# https://community.plotly.com/t/plotly-express-with-xaxis-having-integers-strings/34777

profit_acces_p_2021 = round(sum(atliq_hardware_db.query("segment == 'Accessories' & fiscal_year == 2021").gross_profit) / 1e6, 2)
profit_acces_q_2021 = round(sum(atliq_hardware_db.query("segment == 'Accessories' & fiscal_year == 2021").sold_quantity) / 1e6, 2)
profit_note_p_2021 = round(sum(atliq_hardware_db.query("segment == 'Notebook' & fiscal_year == 2021").gross_profit) / 1e6, 2)
profit_note_q_2021 = round(sum(atliq_hardware_db.query("segment == 'Notebook' & fiscal_year == 2021").sold_quantity) / 1e6, 2)

fig = px.histogram(data_frame = atliq_hardware_db,
                   x = 'segment' , y = 'sold_quantity',
                   color = 'fiscal_year',
                   text_auto = '.5s',
                   width = 950, height = 650,
                   barmode = 'group',
                   title = f'<b>Sales volume in each segments</b><br><sub>In <b>2021</b>, with  <b>{profit_note_q_2021}</b> million quantity <b>notebooks</b> sale made a maximum profit of <b>{profit_note_p_2021}</b> million.</sub>')

fig.update_layout(template = 'plotly_white',
                  xaxis_title = 'Segment',
                  yaxis_title = 'Total sold quantity',
                  legend_title = 'Financial Year',
                  xaxis = {'categoryorder':'total ascending'})

fig.add_annotation(text = f'Sale of <b>{profit_acces_q_2021}</b> million <b>acccessories</b> made a profit of <b>{profit_acces_p_2021}</b> million in 2021<br>',
                   align = 'center', xref = 'paper', yref = 'paper',
                   x = 0.22, y = 0.84, showarrow = False, 
                   font = dict(size = 13, color = 'black'))

fig.write_image('images/plots/sales_volume_segment.png', scale = 3)

fig.show()

![](images/plots/sales_volume_segment.png)

In [ ]:
# https://community.plotly.com/t/plotly-express-with-xaxis-having-integers-strings/34777

img = Image.open('images/curly-braces-u.png')

fhy_2021_share= round(sum(atliq_hardware_db.query('fiscal_year == 2021 & platform == "Brick & Mortar"').sold_quantity) * 100 / sum(atliq_hardware_db.query('fiscal_year == 2021').sold_quantity), 2)
ecom_latin_2021 = round(sum(atliq_hardware_db.query('fiscal_year == 2021 & platform == "E-Commerce" & region == "Latin America"').gross_sales) / 1e6, 2)

fig = px.histogram(data_frame = atliq_hardware_db.query('fiscal_year == 2021'),
                   x = 'region' , y = 'sold_quantity',
                   color  = 'platform',
                   text_auto = '.5s',
                   width = 950, height = 650,
                   barmode = 'group', 
                   title = f'<b>Importance of having physical stores even in <b>2021</b></b><br><sub>Physical stores accounts more-than<b>{fhy_2021_share}%</b> of the total sales quantity.</sub>')

fig.update_layout(xaxis_title = 'Sales region',
                  yaxis_title = 'Total sold quantity',
                  legend_title = 'Platform of Sales',
                  template = 'plotly_white',
                  xaxis = {'categoryorder':'total ascending'})

# https://stackoverflow.com/a/64378982
newnames = {'Brick & Mortar' : 'Physical store', 'E-Commerce': 'E-Commerce'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

fig.add_annotation(text = f'While Latin America prefers <b>E-Commerce</b> over physical stores,<br>and generated <b>{ecom_latin_2021} million</b> revenue via <b>E-Comm.</b>.',
                   align = 'center', xref = 'paper', yref = 'paper',
                   x = 0.1, y = 0.82, showarrow = False, 
                   font = dict(size = 13, color = 'black'))

fig.add_layout_image(dict(source = img, opacity = 0.7,
                          xref = 'paper', yref = 'paper',
                          sizing = 'stretch', layer = 'below'))

fig.update_layout_images(dict(xref = 'paper', yref = 'paper',
                              sizex = 0.17, sizey = 0.05,
                              xanchor ='center', yanchor ='middle',
                              x = 0.12, y = 0.1))

fig.add_shape(type = 'line',
              xref = 'paper', yref = 'paper',
              x0 = 0.12, y0 = 0.13,
              x1 = 0.38, y1 = 0.72)

fig.write_image('images/plots/platform_vs_sales.png', scale = 3)

fig.show()

![](images/plots/platform_vs_sales.png)

In [ ]:
apac_sold_q = round(sum(atliq_hardware_db.query('region == "Asia Pacific"').sold_quantity) / 1e6, 2)

fig = px.pie(data_frame = atliq_hardware_db, values = 'sold_quantity', names = 'region', 
             width = 650, height = 650,
             title = f'<b>Sales quantity market share across regions</b><br><sub><b>Asia Pacific</b> region records major share in portfolio,<br>with <b>{apac_sold_q}</b> million product sales.</sub>')

fig.update_layout(legend_title = 'Regions')
fig.update_traces(textposition = 'auto', textinfo = 'percent+label')

fig.write_image('images/plots/market_share_quantity.png', scale = 3)

fig.show()

![](images/plots/market_share_quantity.png)

In [ ]:
apac_sold_i = round(sum(atliq_hardware_db.query('region == "Asia Pacific"').gross_sales) / 1e9, 2)
latin_sold_i = round(sum(atliq_hardware_db.query('region == "Latin America"').gross_sales) / 1e6, 2)

fig = px.pie(data_frame = atliq_hardware_db, values = 'gross_sales', names = 'region',
             width = 650, height = 650,
             title = f'<b>Gross income share across regions</b><br><sub><b>Asia Pacific</b> region accounts <b>{apac_sold_i}</b> billion income,<br><b>Latin America</b> accounts <b>{latin_sold_i}</b> million.</sub>')

fig.update_layout(legend_title = 'Regions')
fig.update_traces(textposition = 'auto', textinfo = 'percent+label')

fig.write_image('images/plots/market_share_income.png', scale = 3)

fig.show()

![](images/plots/market_share_income.png)

In [ ]:
threshold = 50*1e6

sales_details_2020 = atliq_hardware_db.query('fiscal_year == 2020').groupby('market')[['sold_quantity', 'gross_manufacturing_cost', 'gross_sales', 'gross_profit']].agg(sum).reset_index()
sales_details_2021 = atliq_hardware_db.query('fiscal_year == 2021').groupby('market')[['sold_quantity', 'gross_manufacturing_cost', 'gross_sales', 'gross_profit']].agg(sum).reset_index()

conbined_sales = pd.merge(sales_details_2020, sales_details_2021, on = 'market', suffixes = ['_2020', '_2021'])
fifty_million_club = conbined_sales.query('gross_sales_2020 < @threshold and gross_sales_2021 > @threshold')

In [ ]:
# https://community.plotly.com/t/plotly-express-with-xaxis-having-integers-strings/34777

fig = px.histogram(data_frame = atliq_hardware_db.query('fiscal_year == 2020'),
                   x = 'market' , y = 'gross_sales',
                   text_auto = '.5s',
                   width = 950, height = 660)

fig.update_layout(template = 'plotly_white')
fig.update_layout(xaxis = {'categoryorder':'total ascending'})

fig.show()

![](https://i.imgur.com/rvOQkgi.png)

In [ ]:
# https://community.plotly.com/t/plotly-express-with-xaxis-having-integers-strings/34777

fig = px.histogram(data_frame = sales_details_2021,
                   x = 'market' , y = 'gross_sales',
                   text_auto = '.5s',
                   width = 950, height = 650)

fig.update_layout(template = 'plotly_white')
fig.update_layout(xaxis = {'categoryorder':'total ascending'})

fig.show()

![](https://i.imgur.com/eBBPLCp.png)

In [ ]:
newnames = {'gross_sales_2020' : 'Income in 2020', 'gross_sales_2021' : 'Income in 2021'}

count_ = conbined_sales.query('gross_sales_2020 > gross_sales_2021').market.values[0]
decline_ = round(abs(conbined_sales.query('gross_sales_2020 > gross_sales_2021').gross_sales_2021 - conbined_sales.query('gross_sales_2020 > gross_sales_2021').gross_sales_2020) / 1e6, 2).values[0]

fig = px.histogram(data_frame = fifty_million_club,
                   x = 'market' , y = ['gross_sales_2020', 'gross_sales_2021'],
                   text_auto = '.5s',
                   width = 950, height = 600,
                   barmode = 'group', 
                   title = '<b>Countries / market who joined 50 million income club in 2021</b><br><sub>Market generated less-than <b>50</b> million in <b>2020</b> and generated <b>50+</b> million in <b>2021</b> sales.</sub>')

fig.update_layout(xaxis_title = 'Market',
                  yaxis_title = 'Gross Sales',
                  legend_title = 'Financial Year',
                  template = 'plotly_white',
                  xaxis = {'categoryorder':'total ascending'})

# https://stackoverflow.com/a/64378982
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))

fig.add_annotation(text = f'While <b>{count_}</b> showed a negative decline of<br><b>{decline_}</b> million income in <b>2021</b> compared to 2020.',
                   align = 'center', xref = 'paper', yref = 'paper',
                   x = 0.05, y = 0.95, showarrow = False, 
                   font = dict(size = 13, color = '#665857'))

fig.write_image('images/plots/50_million_club_sales.png', scale = 3)

fig.show()

![](images/plots/50_million_club_sales.png)